### 📖 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현

- 이 체인은 `영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답`해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").

- 항상 세 개의 이모티콘으로 답장하도록 `FewShotPromptTemplate` 또는 `FewShotChatMessagePromptTemplate`을 사용하여 체인에 예시를 제공하세요.

- 메모리가 작동하는지 확인하려면 체인에 `두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청`하세요.

#### 💡 사용할 라이브러리 및 모듈 import 하기

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder 

#### 💡 모델(Model) 생성하기

In [2]:
llm = ChatOpenAI(
    temperature=0.1,        # 창의성 (0 ~ 2)
    model='gpt-3.5-turbo',  # 사용 모델 지정 (Default : gpt-3.5-turbo)
)

C:\Users\SangHyeonLee\AppData\Local\Temp\ipykernel_2456\2442329828.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


#### 💡 FewShot 예제 생성하기

In [3]:
# 영화를 이모지를 바꿔달라는 질문의 예시
movie_to_emoji_examples = [
    {
        "question": "영화 탑건을 이모티콘으로 표현해줘!",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "영화 대부를 이모티콘으로 표현해줘!",
        "answer": "👨‍👨‍👦🔫🍝",
    }
]

#### 💡 메모리 생성

In [4]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,           # 문자열 기반이 아닌, ChatPromptTemplate 에서 사용할 수 있는 형태로 반환
)

C:\Users\SangHyeonLee\AppData\Local\Temp\ipykernel_2456\45786312.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


#### 💡 메모리가 사용되도록 하는 Prompt 생성 

In [5]:
# FewShot 예제를 사용하도록 Prompt 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=['question'],
    examples=movie_to_emoji_examples,
    example_prompt=ChatPromptTemplate.from_messages([
        ('human', '{question}'),
        ('ai', '{answer}')
    ]),
)

# FewShot 을 적용하고, 메모리를 사용하도록 하는 Prompt 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사람과 대화하기 위해 AI의 도움을 받아야 해."),
    MessagesPlaceholder(variable_name='history'),
    few_shot_prompt,
    ("human", "{question}"),
])

#### 💡 메모리를 자동으로 저장하고, 메모리에 접근해 답변을 내놓는 Chain Class 정의

In [6]:
class UseMemoryChain:
    def __init__(self, llm, prompt, memory, input_key="question"):
        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key

        self.chain = RunnablePassthrough.assign(history=self.load_memory) | prompt | llm
        
    def load_memory(self, _):
        return self.memory.load_memory_variables({})['history']

    def invoke(self, question):
        answer = self.chain.invoke({self.input_key: question})
        self.memory.save_context({"input": question}, {"output": answer.content})
        
        return answer.content

#### 💡 UseMemoryChain 객체를 생성

In [7]:
movie_to_emoji_chain = UseMemoryChain(llm, prompt, memory)

##### 📢 첫번째 질문

In [8]:
movie_to_emoji_chain.invoke("영화 명량을 이모지로 표현해줘!")

'🚢💥🌊🇰🇷'

##### 📢 두번째 질문

In [9]:
movie_to_emoji_chain.invoke("영화 미션 임파서블을 이모지로 표현해줘!")

'🕶️💣🔫🚁'

##### 📢 첫번째 질문에서 물어본 영화 이름이 무엇인지 물어보기 (메모리가 적용됐는지 확인)

In [10]:
movie_to_emoji_chain.invoke("내가 첫번째로 물어본 영화 이름이 뭐야 ?")

"네가 먼저 물어본 영화는 '명량'이었어요."